# Task for Today  

***

## Crisis Prediction in African Economies  

Given *data about financial climates in various African countries throughout the years*, let's try to predict whether a **banking crisis** occurred or not in a given year in a given country.

We will use a TensorFlow ANN to make our predictions.

# Getting Started

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [ ]:
data = pd.read_csv('../input/africa-economic-banking-and-systemic-crisis-data/african_crises.csv')

In [ ]:
data

# Preprocessing

Checking if systemic crisis differs from banking crisis in any way.

In [ ]:
((data['banking_crisis'] == 'crisis').astype(int) == data['systemic_crisis']).all()

Check for missing values

In [ ]:
data.isna().sum()

## Encoding

In [ ]:
data = data.drop(['case', 'country'], axis=1)

In [ ]:
data

In [ ]:
cc3_dummies = pd.get_dummies(data['cc3'])
data = pd.concat([data, cc3_dummies], axis=1)
data = data.drop('cc3', axis=1)

In [ ]:
data

In [ ]:
y = data['banking_crisis']
X = data.drop('banking_crisis', axis=1)

In [ ]:
y

In [ ]:
label_encoder = LabelEncoder()

y = label_encoder.fit_transform(y)
{index: label for index, label in enumerate(label_encoder.classes_)}

In [ ]:
y = pd.Series(y).apply(lambda x: 1 - x)

Now `y=1` means we have a crisis and `y=0` means we do not.

## Scaling

In [ ]:
scaler = StandardScaler()

X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

In [ ]:
X

## Splitting

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

# Training

In [ ]:
X.shape

In [ ]:
y.sum() / len(y)

In [ ]:
inputs = tf.keras.Input(shape=(23,))
x = tf.keras.layers.Dense(64, activation='relu')(inputs)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)


model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[tf.keras.metrics.AUC(name="auc")]
)


batch_size = 64
epochs = 60

history = model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=[tf.keras.callbacks.ReduceLROnPlateau()],
    verbose=0
)

# Results

In [ ]:
fig = px.line(
    history.history,
    y=['loss', 'val_loss'],
    labels={'index': "Epoch", 'value': "Loss"},
    title="Training and Validation Loss"
)

fig.show()

In [ ]:
model.evaluate(X_test, y_test)

# Data Every Day  

This notebook is featured on Data Every Day, a YouTube series where I train models on a new dataset each day.  

***

Check it out!  
https://youtu.be/jHrGpb9w4MM